# [PTM] Section 5

## 10. 여러 데이터 소스를 통합 데이터셋으로 합치기
1. 원본 데이터 읽기
2. 전처리

원본 CT 스캔 데이터에 달아놓은 어노테이션 목록으로 훈련 샘플 만들기

### 10.1 원본 CT 데이터 파일
- `.mhd`: 메타데이터 헤더 정보가 포함
- `.raw`: 3차원 배열을 만들 원본 데이터 바이트
- 각 파일 이름은 `시리즈 UID`라고 불리는 CT 스캔 단일 식별자로 시작
  - UID 1.2.3 = 1.2.3.mhd + 1.2.3.raw
- 데이터를 제한하거나 잘라서 모델에 노이즈가 끼지 않게하는 것도 중요

### 10.2 LUNA 애노테이션 데이터 파싱
- LUNA에서 제공하는 `csv` 파일을 먼저 파싱해, 각 CT 스캔 중 관심 있는 부분을 파악할 필요가 있음

In [8]:
import os
import pandas as pd

In [10]:
# 책에서는 bash로 출력하지만, 판다스로 대체
candidates = pd.read_csv(os.path.join(root_path, "candidates.csv"))

candidates.shape

(551065, 5)

In [11]:
candidates.head(3)

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0


In [12]:
# 클래스 분포
# 0: 결절X / 1: 결절O
candidates["class"].value_counts()

0    549714
1      1351
Name: class, dtype: int64

In [13]:
# 결절로 플래그된 후보들에 대한 정보
annotations = pd.read_csv(os.path.join(root_path, "annotations.csv"))

annotations.shape

(1186, 5)

In [14]:
annotations.head(3)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348


#### 10.2.1 훈련셋과 검증셋
- 모든 표준 지도 학습(supervised learning) 작업은 데이터를 훈련셋(training set)과 검증셋(validation set)으로 나눔
- 크기 순으로 정렬한 후, 매 N번째에 대해 검증세트로 구성

#### 10.2.2 어노테이션 데이터와 후보 데이터 합치기


In [16]:
import copy
import csv
import functools
import glob
import os

from collections import namedtuple

import SimpleITK as sitk
import numpy as np

import torch
import torch.cuda
from torch.utils.data import Dataset

In [20]:
# 전문가의 어노테이션을 나타내는, 나름의 인터페이스
# 결절의 상태, 결절의 직겨으 순번, 중심점
CandidateInfoTuple = namedtuple(
    "CandidateInfoTuple",
    "isNodule_bool, diameter_mm, series_uid, center_xyz"
)

In [60]:
# 후보 정보
@functools.lru_cache(1) # 표준 인메모리 캐싱 라이브러리
# - 일부 데이터 파일은 파싱에 시간이 걸리므로, 함수 호출 결과를 메모리에 캐시
# - 인메모리나 온디스크 캐싱을 적절하게 사용하여 데이터 파이프라인 속도를 올려 놓으면 훈련 속도의 개선으로 이어질 수 있음
def getCandidateInfoList(requireOnDisk_bool=True):
    mhd_list = glob.glob(os.path.join(os.path.expanduser("~"), "Downloads/*/*/*.mhd"))
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}
    
    diameter_dict = {}
    with open(os.path.join(root_path, "annotations.csv"), "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])
            
            diameter_dict.setdefault(series_uid, []).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )
            
    # candidates.csv의 정보를 이용해 전체 후보 리스트 만들기
    candidateInfo_list = []
    with open(os.path.join(root_path, "candidates.csv"), "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            
            if series_uid not in presentOnDisk_set and requireOnDisk_bool: continue
            
            isNodule_bool = bool(int(row[4]))
            candidateCenter_xyz = tuple(float(x) for x in row[1:4])
            
            candidateDiameter_mm = 0.0
            for annotation_tup in diameter_dict.get(series_uid, []):
                annotationCenter_xyz, annotationDiameter_mm = annotation_tup
                for i in range(3):
                    delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i])
                    if delta_mm>annotationDiameter_mm/4: # 바운딩 박스 체크
                        break
                    else:
                        candidateDiameter_mm = annotationDiameter_mm
                        break
                
                candidateInfo_list.append(CandidateInfoTuple(isNodule_bool, candidateDiameter_mm, series_uid, candidateCenter_xyz))
    
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [56]:
# 판다스로하면 이렇게 될 듯..?
diameter_dict_df = {}

for idx in range(len(annotations)):
    series_uid = annotations.iloc[idx, 0]
    annotationCenter_xyz = tuple([float(x) for x in annotations.iloc[idx, 1:4]])
    annotationDiameter_mm = float(annotations.iloc[idx, -1])
    
    diameter_dict_df.setdefault(series_uid, []).append(
        (annotationCenter_xyz, annotationDiameter_mm)
    )

### 10.3 개별 CT 스캔 로딩
- 읽어온 CT 데이터를 얻어와 파이썬 객체로 변환해서 3차원 결절 밀도 데이터로 사용할 수 있도록 만드는 작업
- 결절 어노테이션 정보는 원본 데이터에서 얻어내고자 하는 영역에 대한 맵

In [61]:
class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob(os.path.join(os.path.expanduser("~"), f"Downloads/*/*/{series_uid}.mhd"))[0]
        ct_mhd = sitk.ReadImage(mhd_path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

#### 10.3.1 하운스필드 단위

In [62]:
class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob(os.path.join(os.path.expanduser("~"), f"Downloads/*/*/{series_uid}.mhd"))[0]
        ct_mhd = sitk.ReadImage(mhd_path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
        # HU 제거 (시야에 해당하는 값만 남기고, 이외는 모두 제거)
        ct_a.clip(-1000, 1000, ct_a)
        
        self.serires_uid = series_uid
        self.hu_a = ct_a

### 10.4 환자 좌표계를 사용해 결절 위치 정하기
- 통상적으로 모델은 고정된 크기의 입력을 필요로 함 (뉴런 수가 고정되어 있기때문)

#### 10.4.1 환자 좌표계
- 밀리미터 기반 `(X, Y, Z)`를 복셀 주소 기반 `(I, R, C)`로 변환
- `X`는 환자의 왼쪽, `Y`는 뒤쪽(후면), `Z`는 머리(상부)
  - 왼쪽-후면-상부(LPS, left-posterior-superior)
- 해부학적으로 관심있는 위치를 지정하기 위해 사용
- CT 배열과 환자 좌표계 사이의 관계를 정의하는 메타데이터는 파일 헤더에 저장

#### 10.4.2. CT 스캔 형태와 복셀 크기
- 메르카토르(Mercator)식 세계 지도와 유사하게, 실제 비율을 보기 위해서는 비율 계수(scale factor)를 적용

#### 10.4.3 밀리미터를 복셀 주소로 변환하기
